In [1]:
import os
import json
import shutil
import glob
import torch
import cv2
import random
import numpy as np
from sklearn.model_selection import train_test_split
import logging
import pandas as pd

from ultralytics import YOLO

In [2]:
# Check if the MPS (Metal Performance Shaders) backend is available for Apple Silicon devices
if torch.backends.mps.is_available():
    device = "mps"
# If MPS is not available, check if CUDA (NVIDIA GPUs) is available
elif torch.cuda.is_available():
    device = "cuda"
# If neither MPS nor CUDA is available, fall back to using the CPU
else:
    device = "cpu"
# Print the device being used for training
print(f"🚀 Using device: {device}")

🚀 Using device: mps


In [3]:
META_JSON_FILE = 'meta.json'
with open(META_JSON_FILE, "r") as meta_file:
    meta_data = json.load(meta_file)

class_names = [
    "longitudinal crack",
    "transverse crack",
    "alligator crack",
    "block crack",
    "pothole",
    "manhole cover",
    "other corruption"
]
class_map = {name: idx for idx, name in enumerate(class_names)}
print("Loaded class mapping:", class_map)

Loaded class mapping: {'longitudinal crack': 0, 'transverse crack': 1, 'alligator crack': 2, 'block crack': 3, 'pothole': 4, 'manhole cover': 5, 'other corruption': 6}
